Updating the numbers from this story: 

https://highline.huffingtonpost.com/articles/en/poor-millennials/

BOOMER -- 306

MILLENNIAL -- 4459

Hours of minimum wage work needed to pay for four years of public college
Source: National Center for Education Statistics. Calculations based on tuition for four-year public universities from 1973-1976 and 2003-2006.


Note: the most up-to-date tables don't break down colleges vs universities, and there doesn't seem to be any good reason to distinguish between the two in this context, so we will be using all 4-year public institutions with current dollars. 



In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("4_year_public_tuition.csv")
data.head()

,starting_year,tuition_and_fees
0,1963,243.335845
1,1964,255.770493
2,1965,277.922464
3,1966,302.362429
4,1967,310.074436


In [3]:
data_months = data.loc[data.index.repeat(12)].reset_index()
data_months["old_index"] = data_months["index"]
data_months = data_months[["starting_year","tuition_and_fees","old_index"]]
data_months.head()

,starting_year,tuition_and_fees,old_index
0,1963,243.335845,0
1,1963,243.335845,0
2,1963,243.335845,0
3,1963,243.335845,0
4,1963,243.335845,0


In [4]:

data_months["month_count"] = data_months.index%12+1
data_months.head()

,starting_year,tuition_and_fees,old_index,month_count
0,1963,243.335845,0,1
1,1963,243.335845,0,2
2,1963,243.335845,0,3
3,1963,243.335845,0,4
4,1963,243.335845,0,5


In [5]:
data_months["tuition_monthly"] = data_months["tuition_and_fees"]/12
data_months["true_month"] = data_months["month_count"].shift(-5)
data_months["true_year"] = data_months["starting_year"].shift(-5)
data_months.dropna(axis=0, subset=["true_month", "true_year"], inplace=True)
data_months["month_year"] = data_months["true_month"].astype(int).astype(str) + "_" + data_months["true_year"].astype(int).astype(str)
data_months

,starting_year,tuition_and_fees,old_index,month_count,tuition_monthly,true_month,true_year,month_year
0,1963,243.335845,0,1,20.277987,6.0,1963.0,6_1963
1,1963,243.335845,0,2,20.277987,7.0,1963.0,7_1963
2,1963,243.335845,0,3,20.277987,8.0,1963.0,8_1963
3,1963,243.335845,0,4,20.277987,9.0,1963.0,9_1963
4,1963,243.335845,0,5,20.277987,10.0,1963.0,10_1963
5,1963,243.335845,0,6,20.277987,11.0,1963.0,11_1963
6,1963,243.335845,0,7,20.277987,12.0,1963.0,12_1963
7,1963,243.335845,0,8,20.277987,1.0,1964.0,1_1964
8,1963,243.335845,0,9,20.277987,2.0,1964.0,2_1964
9,1963,243.335845,0,10,20.277987,3.0,1964.0,3_1964


In [6]:
minimum_data = pd.read_csv("FEDMINNFRWG.csv")
minimum_data.head()

,DATE,FEDMINNFRWG,month_year
0,10/1/38,0.25,10_1938
1,11/1/38,0.25,11_1938
2,12/1/38,0.25,12_1938
3,1/1/39,0.25,1_1939
4,2/1/39,0.25,2_1939


In [7]:
data_months_combined = pd.merge(data_months, minimum_data, how="left", on="month_year")
data_months_combined.head()

,starting_year,tuition_and_fees,old_index,month_count,tuition_monthly,true_month,true_year,month_year,DATE,FEDMINNFRWG
0,1963,243.335845,0,1,20.277987,6.0,1963.0,6_1963,6/1/63,1.15
1,1963,243.335845,0,2,20.277987,7.0,1963.0,7_1963,7/1/63,1.15
2,1963,243.335845,0,3,20.277987,8.0,1963.0,8_1963,8/1/63,1.15
3,1963,243.335845,0,4,20.277987,9.0,1963.0,9_1963,9/1/63,1.25
4,1963,243.335845,0,5,20.277987,10.0,1963.0,10_1963,10/1/63,1.25


In [8]:
data_months_combined["monthly_hours"] = data_months_combined["tuition_monthly"]/data_months_combined["FEDMINNFRWG"]
data_months_combined.head()

,starting_year,tuition_and_fees,old_index,month_count,tuition_monthly,true_month,true_year,month_year,DATE,FEDMINNFRWG,monthly_hours
0,1963,243.335845,0,1,20.277987,6.0,1963.0,6_1963,6/1/63,1.15,17.633032
1,1963,243.335845,0,2,20.277987,7.0,1963.0,7_1963,7/1/63,1.15,17.633032
2,1963,243.335845,0,3,20.277987,8.0,1963.0,8_1963,8/1/63,1.15,17.633032
3,1963,243.335845,0,4,20.277987,9.0,1963.0,9_1963,9/1/63,1.25,16.222390
4,1963,243.335845,0,5,20.277987,10.0,1963.0,10_1963,10/1/63,1.25,16.222390


In [9]:
data_months_combined["hours_in_last_48_months"] = data_months_combined["monthly_hours"].rolling(48, min_periods=48).sum()
data_months_combined

,starting_year,tuition_and_fees,old_index,month_count,tuition_monthly,true_month,true_year,month_year,DATE,FEDMINNFRWG,monthly_hours,hours_in_last_48_months
0,1963,243.335845,0,1,20.277987,6.0,1963.0,6_1963,6/1/63,1.15,17.633032,NaN
1,1963,243.335845,0,2,20.277987,7.0,1963.0,7_1963,7/1/63,1.15,17.633032,NaN
2,1963,243.335845,0,3,20.277987,8.0,1963.0,8_1963,8/1/63,1.15,17.633032,NaN
3,1963,243.335845,0,4,20.277987,9.0,1963.0,9_1963,9/1/63,1.25,16.222390,NaN
4,1963,243.335845,0,5,20.277987,10.0,1963.0,10_1963,10/1/63,1.25,16.222390,NaN
5,1963,243.335845,0,6,20.277987,11.0,1963.0,11_1963,11/1/63,1.25,16.222390,NaN
6,1963,243.335845,0,7,20.277987,12.0,1963.0,12_1963,12/1/63,1.25,16.222390,NaN
7,1963,243.335845,0,8,20.277987,1.0,1964.0,1_1964,1/1/64,1.25,16.222390,NaN
8,1963,243.335845,0,9,20.277987,2.0,1964.0,2_1964,2/1/64,1.25,16.222390,NaN
9,1963,243.335845,0,10,20.277987,3.0,1964.0,3_1964,3/1/64,1.25,16.222390,NaN


In [10]:
data_months_combined.to_csv("running_monthly.csv")